# ASOS Lookup Function

A user can pass either the ASOS airport code or the “city” and the script/function identifes and returns the name of the HDP station. 
E.g., “I need to find KSAC weather station” or “I need to find the Sacramento Executive Airport weather station” 

Script then identifies KSAC as “ASOSAWOS_XXXXXXXX”

Reasoning: Looking up the specific HDP stations from the attached spreadsheet was not straightforward or easy; users may very likely come to HDP knowing specifically which station they are looking for, and it’s not obvious which station is which.


In [4]:
# imports
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import os
from shapely.geometry import Point
import sys  # Used for progress bar
import datetime
import sys
import os
from io import BytesIO

plt.style.use("tableau-colorblind10")

# Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")  # for lower-level processes

# Set relative paths to other folders and objects in repository.
BUCKET_NAME = "wecc-historical-wx"
QAQC_DIR = "3_qaqc_wx"
MERGE_DIR = "4_merge_wx"
stations_csv_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/all_network_stationlist_merge.csv"

## Final Function

In [12]:
def asos_station_lookup(
    code: str | None = None, city: str | None = None
) -> str:
    """
    Function that returns the HDP station ID for an input of either the
    1. four digit code or (e.g. KSAC)
    2. city (e.g. Sacramento)

    of the airport associated with the station. 

    This functon is specific to the 16 stations of interest shared by project partners.

    Paramters
    ---------
    code : str
        ASOS airport code (e.g. KSAC weather station)
    city : str
        ASOS aiport city (e.g. Sacramento Executive Airport weather station)

    Returns
    -------
    str
        name (ERA ID) of the associated HDP station (e.g. ASOSAWOS_XXXXXXXX)

    """
    # Define dictionaries matching HDP station IDs to airport codes and cities
    ## this was developed beforehand matching HDP ASOSAWOS station locations with those in a list of stations provided by partners

    city_dict = {
        "Arcata": "ASOSAWOS_72594524283",
        "Eureka": "ASOSAWOS_72594524283",
        "Blue Canyon": "ASOSAWOS_72584523225",
        "Burbank": "ASOSAWOS_72288023152",
        "Fresno": "ASOSAWOS_72389093193",
        "Fullerton": "ASOSAWOS_72297603166",
        "Oakland": "ASOSAWOS_72493023230",
        "Palm Springs": "ASOSAWOS_72286893138",
        "Palmdale": "ASOSAWOS_72382023182",
        "Red Bluff": "ASOSAWOS_72591024216",
        "Riverside": "ASOSAWOS_72286903171",
        "Sacramento": "ASOSAWOS_72483023232",
        "San Diego": "ASOSAWOS_72290023188",
        "San Jose": "ASOSAWOS_72494523293",
        "Santa Maria": "ASOSAWOS_72394023273",
        "Santa Rosa": "ASOSAWOS_72495723213",
        "Torrance": "ASOSAWOS_72295503174",
    }

    code_dict = {
        "KACV": "ASOSAWOS_72594524283",
        "KBLU": "ASOSAWOS_72584523225",
        "KBUR": "ASOSAWOS_72288023152",
        "KFAT": "ASOSAWOS_72389093193",
        "KFUL": "ASOSAWOS_72297603166",
        "KOAK": "ASOSAWOS_72493023230",
        "KPSP": "ASOSAWOS_72286893138",
        "KPMD": "ASOSAWOS_72382023182",
        "KRBL": "ASOSAWOS_72591024216",
        "KRAL": "ASOSAWOS_72286903171",
        "KSAC": "ASOSAWOS_72483023232",
        "KSAN": "ASOSAWOS_72290023188",
        "KRHV": "ASOSAWOS_72494523293",
        "KSMX": "ASOSAWOS_72394023273",
        "KSTS": "ASOSAWOS_72495723213",
        "KTOA": "ASOSAWOS_72295503174",
    }

    # If user inputs an airport code
    if code:
        if code in code_dict:
            hdp_station = code_dict[code]
            print(f"The HDP station name for input airport code {code} is {hdp_station}")
        else:
            print(f"Input code '{code}' not in station dictionary.")
    # If user inputs a city
    elif city:
        hdp_station = [val for key, val in city_dict.items() if key in city]
        if hdp_station:
            hdp_station = hdp_station[0]
            print(f"The HDP station name for input airport city '{city}' is {hdp_station}")
        else: 
            print(f"Input city '{city}' not in station dictionary.")
    else:
        print("Please input either an airport code (e.g. code='KSAC') or city (e.g. city='Sacramento').")

    return None

In [13]:
asos_station_lookup(code=None, city="Sacramento Executive Airport")

The HDP station name for input airport city 'Sacramento Executive Airport' is ASOSAWOS_72483023232


In [14]:
asos_station_lookup(code='test', city="Sacramento Executive Airport")

Input code 'test' not in station dictionary.


## Development

I matched up ASOSAWOS stations from the merge station list with the lat lons of the stations in the list that Mithra shared.

In [ ]:
# load station list
station_list = pd.read_csv(stations_csv_path)
station_list = station_list[station_list['network']=='ASOSAWOS']

In [45]:
# convert to geodataframe
asos_stns = gpd.GeoDataFrame(
    station_list,
    geometry=gpd.points_from_xy(
        station_list.longitude, station_list.latitude, crs="EPSG:4326"
    ),
)

In [ ]:
# load in list that Mithra shared (stored in separate repo)
lookup_table = pd.read_csv("../../skillshare_demo/lookup_table.csv")

In [ ]:
# convert it to a geodataframe
lookup_table = gpd.GeoDataFrame(
    lookup_table,
    geometry=gpd.points_from_xy(
        lookup_table.longitude, lookup_table.latitude, crs="EPSG:4326"
    ),
)

In [ ]:
# and filter for ease of use later
lookup_table = lookup_table[
    ["Airport Code", "City", "latitude", "longitude",'geometry']
]

In [ ]:
# establish buffers around the point geometries, as they will not be exact matches

# define the error margin (buffer distance)
error_margin = 0.1  # Adjust this value as needed

# apply buffer to account for the error margin
lookup_table_buffered = lookup_table.copy()
lookup_table_buffered["geometry"] = lookup_table.geometry.buffer(error_margin)

asos_stns_buffered = asos_stns.copy()
asos_stns_buffered["geometry"] = asos_stns.geometry.buffer(error_margin)

/tmp/ipykernel_31410/1684058703.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lookup_table_buffered["geometry"] = lookup_table.geometry.buffer(error_margin)
/tmp/ipykernel_31410/1684058703.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  asos_stns_buffered["geometry"] = asos_stns.geometry.buffer(error_margin)


In [ ]:
# perform the overlay operation (e.g., intersection)
matchup_buffered = gpd.overlay(asos_stns_buffered, lookup_table_buffered, how="intersection")

In [ ]:
# clean things up 
matchup_buffered = matchup_buffered[["era-id", "longitude_1", "latitude_1","longitude_2", "latitude_2",'Airport Code','City']]

Now go through each city in the list (which I added manually before starting this process) and check which HDP station is a match. I added the station ID to a copy of the list in my drive along the way.

In [102]:
matchup_buffered[matchup_buffered["City"] == "San Jose"]

,era-id,longitude_1,latitude_1,longitude_2,latitude_2,Airport Code,City
19,ASOSAWOS_72494523293,-121.924,37.359,-121.82,37.33,KRHV,San Jose


I then downloaded a cleaned version of the table and load it in here (again, stored in a separate repo).

In [ ]:
matchup_table = pd.read_csv("../../skillshare_demo/final_lookup_table.csv")

And generated two dictionaries from it - one for airport codes, another for airport cities

In [123]:
code_dict = pd.Series(matchup_table["HDP station"].values, index=matchup_table["Airport Code"]).to_dict()

In [124]:
city_dict = pd.Series(
    matchup_table["HDP station"].values, index=matchup_table["City"]
).to_dict()

The dictionaries are generated manually in the function itself.